In [3]:
#Step 1 – Import all required libraries
#pip install webdriver_manager
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager

os.chdir('my scrap')

In [4]:
# In this step, we’re installing a Chrome driver and using a headless browser for web scraping.

# In[3]:


#Step 2 – Install Chrome Driver
#Install driver
opts=webdriver.ChromeOptions()
opts.headless=True

driver = webdriver.Chrome(ChromeDriverManager().install() ,options=opts)



[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/dhanishaharjai/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [5]:
#Step 3 – Specify search URL

search_url = "https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"
driver.get(search_url.format(q='Car'))



In [6]:
#Step 4 –  Write a function to take the cursor to the end of the page
def scroll_to_end(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)#sleep_between_interactions



In [7]:
#Step5. Write a function to get URL of each Image

#no license issues

def getImageUrls(name,totalImgs,driver):
    
    search_url = "https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"
    driver.get(search_url.format(q=name))
    img_urls = set()
    img_count = 0
    results_start = 0  
    
    while(img_count<totalImgs): #Extract actual images now
        
        scroll_to_end(driver)
        
        thumbnail_results = driver.find_elements_by_xpath("//img[contains(@class,'Q4LuWd')]")
        totalResults=len(thumbnail_results)
        print(f"Found: {totalResults} search results. Extracting links from{results_start}:{totalResults}")
        
        for img in thumbnail_results[results_start:totalResults]:
            
            img.click()
            time.sleep(2)
            actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                    img_urls.add(actual_image.get_attribute('src'))
            
            img_count=len(img_urls)
            
            if img_count >= totalImgs:
                print(f"Found: {img_count} image links")
                break
            else:
                print("Found:", img_count, "looking for more image links ...")                
                load_more_button = driver.find_element_by_css_selector(".mye4qd")
                driver.execute_script("document.querySelector('.mye4qd').click();")
                results_start = len(thumbnail_results)
    return img_urls



In [8]:
#Step 6: Write a function to download each Image
def downloadImages(folder_path,file_name,url):
    try:
        image_content = requests.get(url).content
    except Exception as e:
        print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
       
        file_path = os.path.join(folder_path, file_name)
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")



In [9]:
#Step7: – Write a function to save each Image in the Destination directory
def saveInDestFolder(searchNames,destDir,totalImgs,driver):
    for name in list(searchNames):
        path=os.path.join(destDir,name)
        if not os.path.isdir(path):
            os.mkdir(path)
        #print('Current Path',path)
        totalLinks=getImageUrls(name,totalImgs,driver)
        print('totalLinks',totalLinks)

        if totalLinks is None:
            print('images not found for :',name)
            continue
        else:
            for i, link in enumerate(totalLinks):
                file_name = f"{i:150}.jpg"
                downloadImages(path,file_name,link)
            
searchNames=['Car','dogs'] 
destDir=f'./Dataset/'
totalImgs=10

saveInDestFolder(searchNames,destDir,totalImgs,driver)



Found: 100 search results. Extracting links from0:100
Found: 1 looking for more image links ...
Found: 2 looking for more image links ...
Found: 3 looking for more image links ...
Found: 3 looking for more image links ...
Found: 4 looking for more image links ...
Found: 5 looking for more image links ...
Found: 6 looking for more image links ...
Found: 7 looking for more image links ...
Found: 8 looking for more image links ...
Found: 9 looking for more image links ...
Found: 10 image links
totalLinks {'https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Sedan-car.svg/1280px-Sedan-car.svg.png', 'https://freesvg.org/img/carsymbol5a.png', 'https://c0.wallpaperflare.com/preview/1002/1002/480/car-concept-vehicle-auto.jpg', 'https://cdn.pixabay.com/photo/2020/11/06/19/11/car-5718701_960_720.jpg', 'https://www.publicdomainpictures.net/pictures/370000/velka/toyota-car.jpg', 'https://cdn.pixabay.com/photo/2017/02/01/12/23/car-2030056_960_720.png', 'https://images.pexels.com/photos/153460